In [ ]:
import boto3

vpc_client = boto3.client('ec2')

In [ ]:
# Note that is you run this in an older account that supports EC2-Classic there might not be 
# a default VPC
response = vpc_client.describe_vpcs()
vpcs = response['Vpcs']
print vpcs

def_vpc = ''
for vpc in vpcs:
    if vpc['IsDefault'] == True:
        def_vpc = vpc['VpcId']
        break
        
print 'default vpc is {}'.format(def_vpc)

In [ ]:
response = vpc_client.describe_subnets(
    Filters=[
        {
            'Name':'vpc-id',
            'Values': [
                def_vpc
            ]
        }
    ]
)

print response

subnets = response['Subnets']

azs = []

for sn in subnets:
    azs.append(sn['AvailabilityZone'])


az_list = ",".join(azs)
print az_list

In [ ]:
# Things to consider when creating a memcached cluster
# a. There is no replication so when a node is lost all the data is lost.
#    More nodes in a cluster means less data is lost when a node crashes
# b. Spreading nodes across availability zones reduces the amount of data
#    lost during an availability zone outage.

In [ ]:
import boto3

client = boto3.client('elasticache')

In [ ]:
# Notes - you need at least two nodes for cross-az mode, and if you provide a list of azs as the
# PreferredAvailabilityZones arguments, the number of nodes must match the size of the availability
# zones list. You can repeat AZs in the availability zone list.
response = client.create_cache_cluster(
    CacheClusterId='sample-cluster-mcd',
    AZMode='cross-az',
    NumCacheNodes=2,
    PreferredAvailabilityZones=azs[0:2],
    CacheNodeType='cache.t2.micro',
    Engine='memcached'
)

print response

In [ ]:
desc = client.describe_cache_clusters(
    CacheClusterId='sample-cluster-mcd'
)

print desc

In [ ]:
# Delete the cluster
response = client.delete_cache_cluster(
    CacheClusterId='sample-cluster-mcd'
)

print response